<a href="https://colab.research.google.com/github/michael-wettach/pythonsamples/blob/main/Python_5_Systemfunktionen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Module zur Interaktion mit dem Betriebssystem</h1>

Frühere Versionen von Python nutzten dafür in erster Linie die Module sys und os. Aktuelle Versionen setzen eher auf das Modul subprocess; alle 3 haben in gewissen Bereichen ihre Berechtigung und sollen hier vorgestellt werden.

<h2>Modul sys</h2>

Das Modul sys stellt Informationen über die Laufzeitumgebung des aktuellen Python-Programms zur Verfügung. 

In [ ]:
import sys
# Informationen über die Laufzeitumgebung
print(sys.version)              # Gibt die aktuelle Python-Version aus
print(sys.version_info)         # Versionsinfos als Objekt mit Attributen
print(sys.platform)             # Betriebssystem, auf dem Python läuft
print(sys.executable)           # Pfad zu Python
print(sys.path)                 # Aktueller Inhalt der path-Variable
print(sys.getrecursionlimit())  # Maximale Rekursionstiefe
sys.setrecursionlimit(500)      # Kann man auch verändern
print(sys.getrecursionlimit())  # Sollte jetzt 500 sein

3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0]
sys.version_info(major=3, minor=7, micro=10, releaselevel='final', serial=0)
linux
/usr/bin/python3
['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']
1000
500


In [ ]:
# Das Modul sys enthält 3 Filehandles sys.stdin, sys.stdout, sys.stderr
# Diese gibt es auch auf Betriebssystem-Ebene in Unix und Windows. 
# Die kann man zur Eingabe und Ausgabe nutzen oder auch umleiten.

print("Textausgabe auf stdout", file=sys.stdout)
sys.stdout.write("So kann man auch Text ausgeben...\n")

# Jetzt leiten wir das Filehandle stdout in eine Datei um
save_stdout = sys.stdout     # für spätere Wiederherstellung

with open("test.txt","w") as fh:
    sys.stdout = fh
    print("Diese Ausgabe wandert in test.txt")

# Hier wird die Umleitung zurück genommen
sys.stdout = save_stdout
print("Alles wieder normal")

Textausgabe auf stdout
So kann man auch Text ausgeben...
Alles wieder normal


<h2>Modul os</h2>

Das Modul os stellt Funktionen für die Interaktion zwischen Python und dem Betriebssystem zur Verfügung. Generell sollte das Modul unabhängig vom Betriebssystem funktionieren, es gibt aber im Modul os Funktionen, die nur auf Linux oder nur auf Windows bereitgestellt werden oder sich geringfügig unterschiedlich verhalten. Das gilt natürlich erst recht für die Betriebssystem-Befehle, die über os-Funktionen aufgerufen werden. Wenn ein Python-Modul auf mehreren Plattformen laufen soll, muss man die Plattform abfragen.

In [ ]:
# Betriebssystem-Aufruf ohne Python
!ls -l

total 8
drwxr-xr-x 1 root root 4096 Jun  1 13:40 sample_data
-rw-r--r-- 1 root root   34 Jun 12 15:23 test.txt


In [ ]:
import sys
import os
if sys.platform == "linux":
    output = os.popen("ls -l")  # führt einen Befehl aus und öffnet ein Handle auf die Ausgabe
    dir = output.readlines()    # readlines() liest aus dem Handle in eine Liste
print(*dir)

total 8
 drwxr-xr-x 1 root root 4096 Jun  1 13:40 sample_data
 -rw-r--r-- 1 root root   34 Jun 12 15:23 test.txt



Python bietet auch die Möglichkeit, einen selbständigen Unterprozess zu starten, der nicht zum Hauptprozess zurückkehrt.<br/> Dafür gibt es die Exec* Funktionen, die ich hier aber nicht weiter erläutere (siehe dazu https://www.python-kurs.eu/forking.php).

<h2>Modul subprocess</h2>

In neueren Python-Versionen wird für Betriebssystem-Aufrufe das Modul subprocess empfohlen. Siehe dazu
* https://docs.python.org/3/library/subprocess.html
* https://www.digitalocean.com/community/tutorials/how-to-use-subprocess-to-run-external-programs-in-python-3-de 

In [ ]:
import subprocess
# Die Funktion check_output erspart das Auslesen des Filehandles 
if sys.platform == "linux":
    output = subprocess.check_output(["ls", "-l"])
# Die Ausgabe ist per default aber im Format byte Array, daher decode()
print(output.decode("utf-8"))    

total 8
drwxr-xr-x 1 root root 4096 Jun  1 13:40 sample_data
-rw-r--r-- 1 root root   34 Jun 12 15:23 test.txt



In [ ]:
# Über Parameter können wir stdin, stdout und encoding setzen
with open("test.txt", "r") as fh:
    output = subprocess.check_output(["grep", "Ausgabe"], stdin=fh, encoding="utf-8")
print(output)

Diese Ausgabe wandert in test.txt



In [ ]:
# Auf Betriebssystem-Ebene möchten wir evtl. auch Pipes nutzen
!dmesg | grep "VFS"

[    0.866703] VFS: Disk quotas dquot_6.6.0
[    0.867533] VFS: Dquot-cache hash table entries: 512 (order 0, 4096 bytes)
[    1.601612] VFS: Mounted root (ext2 filesystem) readonly on device 253:0.


In [ ]:
# Das geht natürlich auch im Python Modul subprocess
# Für das Lesen/Schreiben aus/in pipes wird die Funktion Popen.communicate() empfohlen.

# Für den Unterprozess p1 leiten wir die Standardausgabe in eine Pipe.
p1 = subprocess.Popen(["dmesg"], stdout=subprocess.PIPE)    

# Daraus bedient sich dann der Unterprozess p2 über seine stdin Eingabe. 
# Auch die p2 Ausgabe muss in die Pipe zum anschließenden Auslesen mit communicate().
p2 = subprocess.Popen(["grep", "VFS"], stdin=p1.stdout, stdout=subprocess.PIPE, encoding="utf-8")   

p1.stdout.close()                     # wichtig, damit p1 SIGPIPE bekommt, falls p2 früher endet.

# Hinweis: Statt .stdin.write, .stdout.read or .stderr.read soll communicate() benutzt werden,
# um deadlocks zu vermeiden falls andere OS pipe buffers den Kind-Prozess blockieren.
output, retcode = p2.communicate()    # input=bytearray kann als Parameter auch übergeben werden.
print(output)                         # Mit encoding="utf-8" können wir direkt ausgeben.
print("Returncode: ", retcode)        # retcode < 0 wäre ein Fehler.
p2.stdout.close()

[    0.866703] VFS: Disk quotas dquot_6.6.0
[    0.867533] VFS: Dquot-cache hash table entries: 512 (order 0, 4096 bytes)
[    1.601612] VFS: Mounted root (ext2 filesystem) readonly on device 253:0.

Returncode:  None


In [ ]:
# Beispiel für eine Funktion zum Aufruf eines eigenen Shell Skripts
# und Auswertung von dessen Rückgabe
import re
from subprocess import Popen,PIPE,STDOUT

def shell_call(cmd):
    # takes a shell command and calls it via Popen()
    # result is a tuple (return code, output string)
    args = [p for p in re.split("( |\\\".*?\\\"|'.*?')", cmd) if p.strip()]
    out = Popen(args, stderr=STDOUT, stdout=PIPE, encoding='utf-8')
    result, err = out.communicate()      # liest Textausgabe und Fehlercode
    exit_code = out.wait()               # liest den Shell Exit Code
    return (exit_code, result)

# Wir erzeugen uns zum Testen ein Shell Skript
shell_script = """#!/bin/bash
if [ $# -eq 0 ]; then echo "No parameters given"; exit 0; fi
if [ $# -eq 1 ]; then echo "One parameter given"; exit 1; fi
echo "More than one parameter given"
exit -1
"""

# Und schreiben das in eine Datei
prog = 'my_test.sh'
with open(prog, 'w') as fh:
  print(shell_script, file=fh)

# Dann geben wir dem mal die Ausführungsrechte
rc, text = shell_call('chmod +x /content/' + prog)
print(rc, text)

0 


In [ ]:
# Und jetzt rufen wir das neue Skript auf
rc, text = shell_call('/content/' + prog)
print(rc, text)
rc, text = shell_call('/content/' + prog + ' 1')
print(rc, text)
rc, text = shell_call('/content/' + prog + ' 1 2')
print(rc, text)

0 No parameters given

1 One parameter given

255 More than one parameter given



Das folgende Beispiel zeigt, wie man die Funktionen in einer Testklasse verwenden kann.

In [24]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
import re
import unittest
from subprocess import Popen,PIPE,STDOUT

def shell_call(cmd):
    # takes a shell command and calls it via Popen()
    # result is a tuple (return code, output string list)
    args = [p for p in re.split("( |\\\".*?\\\"|'.*?')", cmd) if p.strip()]
    out = Popen(args, stderr=STDOUT, stdout=PIPE, encoding='utf-8')
    result, err = out.communicate()
    exit_code = out.wait()
    return (exit_code, result)

class MyTests(unittest.TestCase):
    def test_my_test(self, prog='my_test.sh'):
        rc, text = shell_call('./' + prog)
        self.assertEqual(rc, 0)                              # return code == 0
        self.assertRegex(text, '(?m).*No.*')                 # result text contains 'No'

        rc, text = shell_call('./' + prog + ' 1')
        self.assertGreaterEqual(rc, 0)                       # return code >= 0
        self.assertLessEqual(rc, 127)                        # return code <= 127
        self.assertRegex(text, '(?m).*One.*')                # result text contains 'One'

        rc, text = shell_call('./' + prog + ' 1 2')
        self.assertGreaterEqual(rc, 128)                     # return code >= 128
        self.assertRegex(text, '(?m).*More.*')               # result text contains 'More'

# Die Funktion sucht nach Subklassen von TestCase und führt diese aus.
if __name__ == '__main__':
    unittest.main(argv=['first arg is ignored'], exit=False)


.
----------------------------------------------------------------------
Ran 1 test in 0.028s

OK


<h2>Module os.path und pathlib</h2>

Die obigen Module wurden für den Zweck des Testens vorgestellt, so dass ein bereits vorhandenes Programm (egal in welcher Sprache es implementiert ist) aus Python heraus über die Shell aufgerufen und dessen Ergebnis überprüft werden kann. Was ist nun das "Ergebnis" eines aufrufbaren Programms? Vielleicht ein geänderter Tabelleninhalt, eine erzeugte Datei im Filesystem, etc. Im Zusammenhang mit den in Einheit Python_2 vorgestellten Testklassen sollte das Ergebnis in einfacher Weise abfragbar sein, also True/False, A=B etc. 

Die Module os.path und pathlib erlauben es, z. B. die Existenz von Dateien in einfacher Weise zu prüfen, z. B. mit der assertTrue() Funktion.

In [ ]:
import os.path

file_path = 'my_test.sh'
print(os.path.exists(file_path))
print(os.path.isfile(file_path))

file_path = 'sample_data'
print(os.path.exists(file_path))
print(os.path.isfile(file_path))

file_path = 'Bloedsinn.txt'
print(os.path.exists(file_path))
print(os.path.isfile(file_path))

True
True
True
False
False
False


In [ ]:
from pathlib import Path

my_file = Path('my_test.sh')
print(my_file.exists())
print(my_file.is_file())

my_file = Path('sample_data')
print(my_file.exists())
print(my_file.is_file())

my_file = Path('Bloedsinn.txt')
print(my_file.exists())
print(my_file.is_file())

True
True
True
False
False
False


In [ ]:
# Die Module können natürlich noch viel mehr

# Einige os.path Beispiele
# Siehe https://docs.python.org/3/library/os.path.html
import os.path
my_file = './sample_data/readme.md'
my_abs_path = os.path.abspath(my_file)
print(my_abs_path)
print(os.path.basename(my_abs_path))
print(os.path.dirname(my_abs_path))

print()

# Einige pathlib Beispiele
# Siehe https://docs.python.org/3/library/pathlib.html 
from pathlib import Path
my_file = Path('./sample_data/readme.md')
my_abs_path = my_file.resolve()
print(my_abs_path)
print(my_abs_path.name)
print(my_abs_path.parent)
print(my_abs_path.as_uri())

/content/sample_data/readme.md
readme.md
/content/sample_data

/content/sample_data/readme.md
readme.md
/content/sample_data
file:///content/sample_data/readme.md
